In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import trim, lower, col, split

def count_non_matching_districts(oij_path, inec_path):
    """
    Cuenta la cantidad de registros en el conjunto de datos del OIJ 
    que no coinciden con ningún distrito del conjunto de datos del INEC.

    Args:
        oij_path (str): Ruta al archivo CSV del conjunto de datos OIJ.
        inec_path (str): Ruta al archivo CSV del conjunto de datos INEC.

    Returns:
        int: Número de registros del OIJ sin coincidencia en INEC.
    """
    # Crear una sesión de Spark
    spark = SparkSession.builder \
        .appName("Comparar Distritos OIJ e INEC") \
        .getOrCreate()

    # Cargar los datasets
    df_oij = spark.read.csv(oij_path, header=True, inferSchema=True)
    df_inec = spark.read.csv(inec_path, header=True, inferSchema=True)
    
    # Limpiar y transformar la columna de distrito en ambos datasets
    df_oij_clean = df_oij.withColumn("Distrito", trim(lower(col("Distrito"))))
    df_inec_clean = df_inec.withColumn(
        "distrito",
        trim(lower(split(col("Provincia"), ",").getItem(2)))
    )
    
    # Realizar un left anti join para encontrar distritos no coincidentes
    non_matching_districts = df_oij_clean.join(
        df_inec_clean,
        df_oij_clean["Distrito"] == df_inec_clean["distrito"],
        "left_anti"
    )
    
    # Contar registros no coincidentes
    return non_matching_districts.count()


In [ ]:
oij_file_path = "C:\\Users\\andco\\Desktop\\spark\\data\\OIJ.csv"
inec_file_path = "C:\\Users\\andco\\Desktop\\spark\\data\\inec.csv"

# Llamar a la función
resultado = count_non_matching_districts(oij_file_path, inec_file_path)
print(f"Número de registros no coincidentes: {resultado}")
